In [1]:
from importlib import reload
import flymovie as fm
from flymovie.simnuc import Sim
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
from importlib import reload
import scipy.ndimage as ndi
import skimage as ski
%load_ext autoreload
%autoreload 2

In [55]:
#from skimage import regionprops
def extract_nuclei(stack, lmask, box_dims, sample_name, stack_name, outfolder):
    def inbounds(im_shape, halflens, centroid):
        for dim in [1,2]:
            if centroid[dim] - halflens[dim] < 0:
                return False
            if centroid[dim] + halflens[dim] > im_shape[dim]:
                return False
        return True
    
    def get_box(stack, box_halflengths, box_dims, centroid, z):
        box = stack[
                    z:(z + box_dims[0]),
                    (centroid[1] - box_halflengths[1]):(centroid[1] + box_halflengths[1]),
                    (centroid[2] - box_halflengths[2]):(centroid[2] + box_halflengths[2])
                ]
        return box

    regions = ski.measure.regionprops(lmask)
    box_halflengths = (
            int(box_dims[0] / 2),
            int(box_dims[1] / 2),
            int(box_dims[2] / 2), 
        )
    count = 0
    filepaths = []
    ims = []
    for region in regions:
        centroid = ([round(x) for x in region.centroid])
        if inbounds(lmask.shape, box_halflengths, centroid):
            for z in range(lmask.shape[0] - box_dims[0]):
                outfilepath = os.path.join(outfolder, '_'.join([sample_name, stack_name, str(count), str(z)]) + '.pkl')
                im_box = get_box(stack, box_halflengths, box_dims, centroid, z)
                lmask_box = get_box(lmask, box_halflengths, box_dims, centroid, z)
                im_box = np.where(lmask_box == region.label, im_box, 0)
                filepaths.append(outfilepath)
                ims.append(im_box)
            count += 1
    
    return ims, filepaths


def save_nucs(ims, filepaths, bad_nucs):
    for i in range(len(ims)):
        if i not in bad_nucs:
            with open(filepaths[i], 'wb') as outfile:
                pickle.dump(ims[i], outfile)

In [61]:
#stack = fm.read_czi('/Volumes/ELIMBIO/2022-03-13/h2b-gfp-em2-13.czi', swapaxes=False)
#stack = fm.read_czi('/Volumes/ELIMBIO/2022-03-13/h2b-gfp-em2-14.czi', swapaxes=False)
#stack = fm.read_czi('/Volumes/ELIMBIO/2022-03-13/h2b-gfp-em2-15.czi', swapaxes=False)
#stack = fm.read_czi('/Volumes/ELIMBIO/2022-03-13/rpb1-gfp-em2-06.czi', swapaxes=False)
#stack = fm.read_czi('/Volumes/ELIMBIO/2022-03-13/rpb1-gfp-em2-07.czi', swapaxes=False)
#stack = fm.read_czi('/Volumes/ELIMBIO/2022-03-13/rpb1-gfp-em2-09.czi', swapaxes=False)
#stack = fm.read_czi('/Volumes/ELIMBIO/2022-03-13/zld-mneon-em1-01.czi', swapaxes=False)
#stack = fm.read_czi('/Volumes/ELIMBIO/2022-03-13/zld-mneon-em1-02.czi', swapaxes=False)
stack = fm.read_czi('/Volumes/ELIMBIO/2022-03-13/zld-mneon-em1-03.czi', swapaxes=False)
lmask = fm.segment_nuclei_3Dstack_rpb1(stack)
lmask = fm.labelmask_filter_objsize(lmask, 100, 1e5)
fm.viewer(lmask, 5)

interactive(children=(Dropdown(description='Color', index=1, options=('gray_r', 'cividis', 'inferno', 'plasma'…

In [62]:
ims, filepaths = extract_nuclei(stack, lmask, (34,100,100), 'zld', 'em1-03', '/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/real_nuclei')
fm.viewer(np.array(ims).max(axis=1), 5)

interactive(children=(Dropdown(description='Color', index=1, options=('gray_r', 'cividis', 'inferno', 'plasma'…

In [60]:
bad_nucs = []
save_nucs(ims, filepaths, bad_nucs)